In [1]:
import json
import numpy as np
import faiss
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
import glob
from dataclasses import dataclass
import pickle
from pathlib import Path

In [2]:
@dataclass
class Solution:
    title: str
    solution: str
    difficulty: str
    topics: str
    companies: str

In [3]:
class LeetCodeEmbedder:
    def __init__(
        self,
        model_name: str = "all-MiniLM-L6-v2",
        index_path: str = "leetcode_hnsw2.index",
        metadata_path: str = "leetcode_metadata2.pkl"
    ):
        self.encoder = SentenceTransformer(model_name)
        self.index_path = Path(index_path)
        self.metadata_path = Path(metadata_path)
        self.solutions: List[Solution] = []
        self.dimension = 384  # MiniLM-L6-v2 dimension

    def process_solution(self, json_data: dict) -> str:
        """Extract and clean solution text from JSON"""
        solution = json_data.get('solution', '')
        # Remove markdown code blocks
        solution = solution.replace('```python', '').replace('```', '')
        # Extract approach and implementation details
        if 'Approach:' in solution:
            solution = solution.split('Approach:')[1]
        return solution.strip()

    def load_and_embed_solutions(self, json_dir: str):
        """Load solutions from JSON files and create embeddings"""
        # Load all JSON files
        json_files = glob.glob(f"{json_dir}/*.json")

        # Process each solution
        for file_path in json_files:
            with open(file_path, 'r') as f:
                data = json.load(f)

                solution = Solution(
                    title=data['title'],
                    solution=self.process_solution(data),
                    difficulty=data['difficulty'],
                    topics=data['topics'],
                    companies=data['companies']
                )
                self.solutions.append(solution)

        # Create composite texts for embedding
        texts = [
            f"Title: {sol.title}\nDifficulty: {sol.difficulty}\n"
            f"Topics: {sol.topics}\nSolution: {sol.solution}"
            for sol in self.solutions
        ]

        # Generate embeddings
        print("Generating embeddings...")
        embeddings = self.encoder.encode(texts, show_progress_bar=True)

        # Create and train HNSW index
        print("Creating HNSW index...")
        index = faiss.IndexHNSWFlat(
            self.dimension, 32)  # 32 neighbors per layer
        index.hnsw.efConstruction = 40  # Higher accuracy during construction
        index.hnsw.efSearch = 32  # Higher accuracy during search

        # Add vectors to index
        index.add(embeddings.astype(np.float32))

        # Save index and metadata
        print("Saving index and metadata...")
        faiss.write_index(index, str(self.index_path))
        with open(self.metadata_path, 'wb') as f:
            pickle.dump(self.solutions, f)

        print(f"Processed {len(self.solutions)} solutions")


In [4]:
class LeetCodeRetriever:
    def __init__(
        self,
        index_path: str = "leetcode_hnsw.index",
        metadata_path: str = "leetcode_metadata.pkl",
        model_name: str = "all-MiniLM-L6-v2"
    ):
        self.encoder = SentenceTransformer(model_name)
        self.index = faiss.read_index(index_path)

        with open(metadata_path, 'rb') as f:
            self.solutions = pickle.load(f)

    def search(
        self,
        query: str,
        k: int = 3,
        return_scores: bool = True
    ) -> List[Tuple[Solution, float]]:
        """Search for similar solutions"""
        # Encode query
        query_vector = self.encoder.encode([query])

        # Search index
        distances, indices = self.index.search(
            query_vector.astype(np.float32), k
        )

        # Return results
        if return_scores:
            return [
                (self.solutions[idx], float(score))
                for idx, score in zip(indices[0], distances[0])
            ]
        return [self.solutions[idx] for idx in indices[0]]

    def get_topic_similar(
        self,
        topic: str,
        k: int = 5
    ) -> List[Solution]:
        """Find solutions with similar topics"""
        return [
            sol for sol in self.solutions
            if topic.lower() in sol.topics.lower()
        ][:k]



In [4]:
embedder = LeetCodeEmbedder()
embedder.load_and_embed_solutions(
    "research/leetcode_solutions")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Embedder initialized successfully.


In [5]:
retriever = LeetCodeRetriever()

In [6]:
queries = [
    "matrix distance calculation",
    "dynamic programming coin change",
    "tree traversal problem"
]

In [7]:
for query in queries:
    print(f"\nQuery: {query}")
    results = retriever.search(query, k=2)
    for solution, score in results:
        print(f"\nTitle: {solution.title}")
        print(f"Score: {score:.3f}")
        print(f"Topics: {solution.topics}")
        print(f"Difficulty: {solution.difficulty}")


Query: matrix distance calculation

Title: 01 Matrix
Score: 1.005
Topics: Depth-first Search,Breadth-first Search
Difficulty: Medium

Title: Matrix Cells in Distance Order
Score: 1.007
Topics: Sort
Difficulty: Easy

Query: dynamic programming coin change

Title: Coin Change
Score: 0.598
Topics: Dynamic Programming
Difficulty: Medium

Title: Coin Change 2
Score: 0.713
Topics: nan
Difficulty: Medium

Query: tree traversal problem

Title: Add One Row to Tree
Score: 0.758
Topics: Tree
Difficulty: Medium

Title: Inorder Successor in BST
Score: 0.773
Topics: Tree
Difficulty: Medium


In [3]:
from dataclasses import dataclass
from typing import List, Optional


@dataclass
class TestQuery:
    query: str
    expected_problems: List[str]  # List of relevant problem titles
    category: str
    difficulty: str
    challenge_type: str

In [4]:
class LeetCodeTestQueries:
    @staticmethod
    def get_conceptually_ambiguous_queries() -> List[TestQuery]:
        """Queries that are conceptually ambiguous and could match multiple problems"""
        return [
            TestQuery(
                query="find minimum operations to make array beautiful",
                expected_problems=["Minimum Operations to Make Array Beautiful",
                                   "Make Array Zero by Subtracting Equal Amounts"],
                category="array_manipulation",
                difficulty="hard",
                challenge_type="conceptual_ambiguity"
            ),
            TestQuery(
                query="optimize path finding in grid with obstacles using minimum jumps",
                expected_problems=["Jump Game", "Jump Game II",
                                   "Shortest Path in Binary Matrix"],
                category="dynamic_programming",
                difficulty="hard",
                challenge_type="algorithm_overlap"
            ),
            TestQuery(
                query="find maximum sum subarray with alternating positive negative elements",
                expected_problems=["Maximum Subarray",
                                   "Maximum Alternating Subsequence Sum"],
                category="array",
                difficulty="medium",
                challenge_type="requirement_ambiguity"
            ),
            TestQuery(
                query="optimize tree traversal with minimum memory usage without recursion",
                expected_problems=["Binary Tree Inorder Traversal",
                                   "Morris Traversal", "Iterative Tree Traversal"],
                category="trees",
                difficulty="medium",
                challenge_type="implementation_ambiguity"
            ),
            TestQuery(
                query="find shortest cycle in undirected graph with weighted edges",
                expected_problems=["Shortest Cycle in Graph",
                                   "Minimum Spanning Tree", "Network Delay Time"],
                category="graphs",
                difficulty="hard",
                challenge_type="algorithm_complexity"
            )
        ]

    @staticmethod
    def get_semantically_challenging_queries() -> List[TestQuery]:
        """Queries using non-standard terminology or domain-specific language"""
        return [
            TestQuery(
                query="implement memcache with LRU and time-based eviction",
                expected_problems=[
                    "LRU Cache", "Design In-Memory File System", "Time Based Key-Value Store"],
                category="system_design",
                difficulty="hard",
                challenge_type="domain_specific"
            ),
            TestQuery(
                query="optimize matrix chain multiplication using bottom-up approach",
                expected_problems=[
                    "Matrix Chain Multiplication", "Burst Balloons"],
                category="dynamic_programming",
                difficulty="hard",
                challenge_type="technical_terminology"
            ),
            TestQuery(
                query="implement concurrent rate limiter with sliding window",
                expected_problems=[
                    "Rate Limiter", "Sliding Window Maximum", "Design Hit Counter"],
                category="system_design",
                difficulty="hard",
                challenge_type="cross_domain"
            ),
            TestQuery(
                query="find strongly connected components in directed graph without Tarjan",
                expected_problems=[
                    "Critical Connections in a Network", "Strongly Connected Components"],
                category="graphs",
                difficulty="hard",
                challenge_type="algorithm_constraint"
            ),
            TestQuery(
                query="optimize database index structure using B+ tree with minimum memory",
                expected_problems=["Design In-Memory File System",
                                   "Design Search Autocomplete System"],
                category="system_design",
                difficulty="hard",
                challenge_type="system_specific"
            )
        ]

    @staticmethod
    def get_edge_case_queries() -> List[TestQuery]:
        """Queries testing edge cases and boundary conditions"""
        return [
            TestQuery(
                query="handle concurrent modifications in binary search tree with duplicates",
                expected_problems=[
                    "Binary Search Tree Iterator", "Serialize and Deserialize BST"],
                category="trees",
                difficulty="hard",
                challenge_type="concurrency_edge"
            ),
            TestQuery(
                query="optimize space complexity for computing large fibonacci numbers with modulo",
                expected_problems=["Fibonacci Number",
                                   "Matrix Exponentiation"],
                category="math",
                difficulty="medium",
                challenge_type="optimization_edge"
            ),
            TestQuery(
                query="handle overflow in binary tree with maximum path sum allowing negative values",
                expected_problems=[
                    "Binary Tree Maximum Path Sum", "Path Sum II"],
                category="trees",
                difficulty="hard",
                challenge_type="numeric_edge"
            ),
            TestQuery(
                query="find minimum cuts in flow network with negative capacity edges",
                expected_problems=["Network Flow",
                                   "Minimum Cut", "Maximum Flow"],
                category="graphs",
                difficulty="hard",
                challenge_type="constraint_edge"
            ),
            TestQuery(
                query="optimize palindrome checking for very large strings with unicode characters",
                expected_problems=["Valid Palindrome",
                                   "Longest Palindromic Substring"],
                category="strings",
                difficulty="medium",
                challenge_type="input_edge"
            )
        ]


In [5]:
def test_retrieval_system(retriever, k: int = 5):
    test_queries = LeetCodeTestQueries()

    all_query_sets = [
        ("Conceptually Ambiguous", test_queries.get_conceptually_ambiguous_queries()),
        ("Semantically Challenging",
         test_queries.get_semantically_challenging_queries()),
        ("Edge Cases", test_queries.get_edge_case_queries())
    ]

    for set_name, queries in all_query_sets:
        print(f"\nTesting {set_name} Queries:")
        print("-" * 50)

        for query in queries:
            results = retriever.search(query.query, k=k)

            print(f"\nQuery: {query.query}")
            print(f"Challenge Type: {query.challenge_type}")
            print(f"Expected Problems: {', '.join(query.expected_problems)}")
            print("\nTop Results:")
            for solution, score in results:
                print(f"- {solution.title} (Score: {score:.3f})")
            print()



In [11]:
test_retrieval_system(retriever)


Testing Conceptually Ambiguous Queries:
--------------------------------------------------

Query: find minimum operations to make array beautiful
Challenge Type: conceptual_ambiguity
Expected Problems: Minimum Operations to Make Array Beautiful, Make Array Zero by Subtracting Equal Amounts

Top Results:
- Shortest Subarray with Sum at Least K (Score: 0.855)
- Sum of Subarray Minimums (Score: 0.861)
- Minimum Size Subarray Sum (Score: 0.870)
- Find Minimum in Rotated Sorted Array (Score: 0.888)
- Decrease Elements To Make Array Zigzag (Score: 0.923)


Query: optimize path finding in grid with obstacles using minimum jumps
Challenge Type: algorithm_overlap
Expected Problems: Jump Game, Jump Game II, Shortest Path in Binary Matrix

Top Results:
- Unique Paths II (Score: 0.709)
- Unique Paths III (Score: 0.729)
- Minimum Path Sum (Score: 0.857)
- Minimum Moves to Reach Target with Rotations (Score: 0.875)
- Shortest Path in Binary Matrix (Score: 0.879)


Query: find maximum sum subarray 

In [12]:
import faiss
import numpy as np
import time
from typing import Tuple, List

In [13]:
class HNSWTuner:
    def __init__(self, embeddings: np.ndarray):
        self.embeddings = embeddings
        self.dimension = embeddings.shape[1]

    def tune_parameters(self) -> Tuple[int, int, int]:
        """Find optimal HNSW parameters"""
        # Test different M (max number of connections)
        M_values = [16, 32, 64]
        # Test different efConstruction (size of dynamic candidate list)
        efC_values = [40, 80, 120]
        # Test different efSearch (size of dynamic candidate list during search)
        efS_values = [32, 64, 128]

        best_params = None
        best_qps = 0  # Queries per second

        for M in M_values:
            for efC in efC_values:
                for efS in efS_values:
                    # Create index with current parameters
                    index = faiss.IndexHNSWFlat(self.dimension, M)
                    index.hnsw.efConstruction = efC
                    index.hnsw.efSearch = efS

                    # Train and add vectors
                    index.add(self.embeddings)

                    # Benchmark search performance
                    qps = self._benchmark_search(index)

                    print(f"M={M}, efC={efC}, efS={efS}: {qps:.2f} queries/sec")

                    if qps > best_qps:
                        best_qps = qps
                        best_params = (M, efC, efS)

        return best_params

    def _benchmark_search(self, index: faiss.IndexHNSW, n_queries: int = 100) -> float:
        """Benchmark search performance"""
        # Generate random queries
        queries = np.random.randn(n_queries, self.dimension).astype('float32')

        start_time = time.time()
        for query in queries:
            _ = index.search(query.reshape(1, -1), k=5)
        end_time = time.time()

        return n_queries / (end_time - start_time)


def create_optimized_hnsw_index(embeddings: np.ndarray) -> faiss.IndexHNSW:
    """Create HNSW index with optimized parameters"""
    # First tune parameters
    tuner = HNSWTuner(embeddings)
    M, efC, efS = tuner.tune_parameters()

    # Create index with optimal parameters
    dimension = embeddings.shape[1]
    index = faiss.IndexHNSWFlat(dimension, M)
    index.hnsw.efConstruction = efC
    index.hnsw.efSearch = efS

    # Add vectors
    index.add(embeddings)

    return index




In [14]:
class OptimizedRetriever:
    def __init__(self, embeddings: np.ndarray, problems: List[dict]):
        self.index = create_optimized_hnsw_index(embeddings)
        self.problems = problems

    def search(self, query: str, k: int = 5) -> List[Tuple[dict, float]]:
        # Convert query to embedding
        query_vector = self._get_embedding(query)

        # Search with optimized parameters
        distances, indices = self.index.search(
            query_vector.reshape(1, -1).astype('float32'),
            k
        )

        return [
            (self.problems[idx], float(dist))
            for idx, dist in zip(indices[0], distances[0])
        ]

    def _get_embedding(self, text: str) -> np.ndarray:
        # Implement your embedding logic here
        pass




In [15]:
dimension = 384 
n_vectors = 1000  

embeddings = np.random.randn(n_vectors, dimension).astype('float32')


index = create_optimized_hnsw_index(embeddings)
print("Optimized HNSW index created")

M=16, efC=40, efS=32: 24759.76 queries/sec
M=16, efC=40, efS=64: 23095.12 queries/sec
M=16, efC=40, efS=128: 14677.72 queries/sec
M=16, efC=80, efS=32: 35463.80 queries/sec
M=16, efC=80, efS=64: 23418.78 queries/sec
M=16, efC=80, efS=128: 14518.19 queries/sec
M=16, efC=120, efS=32: 35484.81 queries/sec
M=16, efC=120, efS=64: 22518.54 queries/sec
M=16, efC=120, efS=128: 14453.15 queries/sec
M=32, efC=40, efS=32: 25151.74 queries/sec
M=32, efC=40, efS=64: 19055.49 queries/sec
M=32, efC=40, efS=128: 13155.71 queries/sec
M=32, efC=80, efS=32: 31270.44 queries/sec
M=32, efC=80, efS=64: 20695.24 queries/sec
M=32, efC=80, efS=128: 13360.21 queries/sec
M=32, efC=120, efS=32: 29985.02 queries/sec
M=32, efC=120, efS=64: 20392.38 queries/sec
M=32, efC=120, efS=128: 13204.58 queries/sec
M=64, efC=40, efS=32: 21009.34 queries/sec
M=64, efC=40, efS=64: 18433.26 queries/sec
M=64, efC=40, efS=128: 12520.31 queries/sec
M=64, efC=80, efS=32: 24834.53 queries/sec
M=64, efC=80, efS=64: 17972.76 queries/se

In [26]:
index.hnsw.efConstruction, index.hnsw.efSearch

(120, 32)

In [6]:
class LeetCodeEmbedder:
    def __init__(
        self,
        model_name: str = "all-MiniLM-L6-v2",
        index_path: str = "leetcode_hnsw_optimal.index",
        metadata_path: str = "leetcode_metadata_optimal.pkl"
    ):
        self.encoder = SentenceTransformer(model_name)
        self.index_path = Path(index_path)
        self.metadata_path = Path(metadata_path)
        self.solutions: List[Solution] = []
        self.dimension = 384  # MiniLM-L6-v2 dimension

    def process_solution(self, json_data: dict) -> str:
        """Extract and clean solution text from JSON"""
        solution = json_data.get('solution', '')
        # Remove markdown code blocks
        solution = solution.replace('```python', '').replace('```', '')
        # Extract approach and implementation details
        if 'Approach:' in solution:
            solution = solution.split('Approach:')[1]
        return solution.strip()

    # def load_and_embed_solutions(self, json_dir: str):
    #     """Load solutions from JSON files and create embeddings"""
    #     # Load all JSON files
    #     json_files = glob.glob(f"{json_dir}/*.json")

        # # Process each solution
        # for file_path in json_files:
        #     with open(file_path, 'r') as f:
        #         data = json.load(f)

        #         solution = Solution(
        #             title=data['title'],
        #             solution=self.process_solution(data),
        #             difficulty=data['difficulty'],
        #             topics=data['topics'],
        #             companies=data['companies']
        #         )
        #         self.solutions.append(solution)

        # # Create composite texts for embedding
        # texts = [
        #     f"Title: {sol.title}\nDifficulty: {sol.difficulty}\n"
        #     f"Topics: {sol.topics}\nSolution: {sol.solution}"
        #     for sol in self.solutions
        # ]

        # # Generate embeddings
        # print("Generating embeddings...")
        # embeddings = self.encoder.encode(texts, show_progress_bar=True)

        # # Create and train HNSW index
        # print("Creating HNSW index...")
        # index = faiss.IndexHNSWFlat(
        #     self.dimension, 64)  # 32 neighbors per layer
        # index.hnsw.efConstruction = 120  # Higher accuracy during construction
        # index.hnsw.efSearch = 32  # Higher accuracy during search

        # # Add vectors to index
        # index.add(embeddings.astype(np.float32))

        # # Save index and metadata
        # print("Saving index and metadata...")
        # faiss.write_index(index, str(self.index_path))
        # with open(self.metadata_path, 'wb') as f:
        #     pickle.dump(self.solutions, f)

        # print(f"Processed {len(self.solutions)} solutions")


    def load_and_embed_solutions(self, json_dir: str):
        """Load solutions from JSON files and create embeddings."""
        # Load all JSON files
        json_files = glob.glob(f"{json_dir}/*.json")
        if not json_files:
            print(f"No JSON files found in directory: {json_dir}")
            return

        # Process each solution
        for file_path in json_files:
            with open(file_path, 'r') as f:
                data = json.load(f)

                solution = Solution(
                    title=data['title'],
                    solution=self.process_solution(data),
                    difficulty=data['difficulty'],
                    topics=data['topics'],
                    companies=data['companies']
                )
                self.solutions.append(solution)

        # Create composite texts for embedding
        texts = [
            f"Title: {sol.title}\nDifficulty: {sol.difficulty}\n"
            f"Topics: {sol.topics}\nSolution: {sol.solution}"
            for sol in self.solutions
        ]

        # Debugging: Check if texts list is empty
        if not texts:
            print("No texts generated for embedding. Check the JSON files.")
            return

        # Generate embeddings
        print("Generating embeddings...")
        embeddings = self.encoder.encode(texts, show_progress_bar=True)

        # Debugging: Check the shape of embeddings
        print(f"Shape of embeddings: {embeddings.shape}")
        if len(embeddings.shape) == 1:
            print("Warning: Embeddings are 1D. Reshaping to 2D.")
            embeddings = embeddings.reshape(1, -1)  # Reshape to (1, embedding_dim)

        # Create and train HNSW index
        print("Creating HNSW index...")
        index = faiss.IndexHNSWFlat(self.dimension, 64)  # 64 neighbors per layer
        index.hnsw.efConstruction = 120  # Higher accuracy during construction
        index.hnsw.efSearch = 32  # Higher accuracy during search

        # Add vectors to index
        print("Adding embeddings to index...")
        index.add(embeddings.astype(np.float32))

        # Save index and metadata
        print("Saving index and metadata...")
        faiss.write_index(index, str(self.index_path))
        with open(self.metadata_path, 'wb') as f:
            pickle.dump(self.solutions, f)

        print(f"Processed {len(self.solutions)} solutions")

In [7]:
class LeetCodeRetriever:
    def __init__(
        self,
        index_path: str = "leetcode_hnsw_optimal.index",
        metadata_path: str = "leetcode_metadata_optimal.pkl",
        model_name: str = "all-MiniLM-L6-v2"
    ):
        self.encoder = SentenceTransformer(model_name)
        self.index = faiss.read_index(index_path)

        with open(metadata_path, 'rb') as f:
            self.solutions = pickle.load(f)

    def search(
        self,
        query: str,
        k: int = 3,
        return_scores: bool = True
    ) -> List[Tuple[Solution, float]]:
        """Search for similar solutions"""
        # Encode query
        query_vector = self.encoder.encode([query])

        # Search index
        distances, indices = self.index.search(
            query_vector.astype(np.float32), k
        )

        # Return results
        if return_scores:
            return [
                (self.solutions[idx], float(score))
                for idx, score in zip(indices[0], distances[0])
            ]
        return [self.solutions[idx] for idx in indices[0]]

    def get_topic_similar(
        self,
        topic: str,
        k: int = 5
    ) -> List[Solution]:
        """Find solutions with similar topics"""
        return [
            sol for sol in self.solutions
            if topic.lower() in sol.topics.lower()
        ][:k]

In [30]:
embedder = LeetCodeEmbedder()
embedder.load_and_embed_solutions(
    "/Users/uditrawat/Desktop/RAG-Project/research/leetcode_solutions")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


No JSON files found in directory: /Users/uditrawat/Desktop/RAG-Project/research/leetcode_solutions


In [8]:
retriever_optimal = LeetCodeRetriever()

In [9]:
test_retrieval_system(retriever_optimal)


Testing Conceptually Ambiguous Queries:
--------------------------------------------------

Query: find minimum operations to make array beautiful
Challenge Type: conceptual_ambiguity
Expected Problems: Minimum Operations to Make Array Beautiful, Make Array Zero by Subtracting Equal Amounts

Top Results:
- Shortest Subarray with Sum at Least K (Score: 0.855)
- Sum of Subarray Minimums (Score: 0.861)
- Minimum Size Subarray Sum (Score: 0.870)
- Find Minimum in Rotated Sorted Array (Score: 0.888)
- Decrease Elements To Make Array Zigzag (Score: 0.923)


Query: optimize path finding in grid with obstacles using minimum jumps
Challenge Type: algorithm_overlap
Expected Problems: Jump Game, Jump Game II, Shortest Path in Binary Matrix

Top Results:
- Unique Paths II (Score: 0.709)
- Unique Paths III (Score: 0.729)
- Minimum Path Sum (Score: 0.857)
- Minimum Moves to Reach Target with Rotations (Score: 0.875)
- Shortest Path in Binary Matrix (Score: 0.879)


Query: find maximum sum subarray 

In [ ]:
# Testing Conceptually Ambiguous Queries:
# --------------------------------------------------

# Query: find minimum operations to make array beautiful
# Challenge Type: conceptual_ambiguity
# Expected Problems: Minimum Operations to Make Array Beautiful, Make Array Zero by Subtracting Equal Amounts

# Top Results:
# - Shortest Subarray with Sum at Least K (Score: 0.855)
# - Sum of Subarray Minimums (Score: 0.861)
# - Minimum Size Subarray Sum (Score: 0.870)
# - Find Minimum in Rotated Sorted Array (Score: 0.888)
# - Decrease Elements To Make Array Zigzag (Score: 0.923)


# Query: optimize path finding in grid with obstacles using minimum jumps
# Challenge Type: algorithm_overlap
# Expected Problems: Jump Game, Jump Game II, Shortest Path in Binary Matrix

# Top Results:
# - Unique Paths II (Score: 0.709)
# - Unique Paths III (Score: 0.729)
# - Minimum Path Sum (Score: 0.857)
# - Minimum Moves to Reach Target with Rotations (Score: 0.875)
# - Shortest Path in Binary Matrix (Score: 0.879)


# Query: find maximum sum subarray with alternating positive negative elements
# Challenge Type: requirement_ambiguity
# Expected Problems: Maximum Subarray, Maximum Alternating Subsequence Sum

# Top Results:
# - Maximum Subarray (Score: 0.581)
# - Maximum Sum Circular Subarray (Score: 0.712)
# - Partition Array for Maximum Sum (Score: 0.726)
# - Maximum Size Subarray Sum Equals k (Score: 0.742)
# - Maximum Sum of 3 Non-Overlapping Subarrays (Score: 0.754)


# Query: optimize tree traversal with minimum memory usage without recursion
# Challenge Type: implementation_ambiguity
# Expected Problems: Binary Tree Inorder Traversal, Morris Traversal, Iterative Tree Traversal

# Top Results:
# - Binary Tree Postorder Traversal (Score: 0.732)
# - Maximum Depth of Binary Tree (Score: 0.771)
# - Binary Tree Level Order Traversal II (Score: 0.802)
# - Tree Node (Score: 0.807)
# - Increasing Order Search Tree (Score: 0.818)


# Query: find shortest cycle in undirected graph with weighted edges
# Challenge Type: algorithm_complexity
# Expected Problems: Shortest Cycle in Graph, Minimum Spanning Tree, Network Delay Time

# Top Results:
# - Shortest Path Visiting All Nodes (Score: 0.988)
# - Network Delay Time (Score: 1.040)
# - Cheapest Flights Within K Stops (Score: 1.058)
# - All Paths from Source Lead to Destination (Score: 1.068)
# - Shortest Path with Alternating Colors (Score: 1.070)


# Testing Semantically Challenging Queries:
# --------------------------------------------------

# Query: implement memcache with LRU and time-based eviction
# Challenge Type: domain_specific
# Expected Problems: LRU Cache, Design In-Memory File System, Time Based Key-Value Store

# Top Results:
# - LFU Cache (Score: 1.070)
# - LRU Cache (Score: 1.151)
# - Design Bounded Blocking Queue (Score: 1.252)
# - Building H2O (Score: 1.361)
# - Design Circular Queue (Score: 1.404)


# Query: optimize matrix chain multiplication using bottom-up approach
# Challenge Type: technical_terminology
# Expected Problems: Matrix Chain Multiplication, Burst Balloons

# Top Results:
# - Sparse Matrix Multiplication (Score: 1.064)
# - Range Addition II (Score: 1.133)
# - Longest Increasing Path in a Matrix (Score: 1.135)
# - Set Matrix Zeroes (Score: 1.154)
# - Search a 2D Matrix II (Score: 1.158)


# Query: implement concurrent rate limiter with sliding window
# Challenge Type: cross_domain
# Expected Problems: Rate Limiter, Sliding Window Maximum, Design Hit Counter

# Top Results:
# - Sliding Window Maximum (Score: 1.122)
# - Design Bounded Blocking Queue (Score: 1.130)
# - Open the Lock (Score: 1.216)
# - Moving Average from Data Stream (Score: 1.257)
# - Max Consecutive Ones III (Score: 1.271)


# Query: find strongly connected components in directed graph without Tarjan
# Challenge Type: algorithm_constraint
# Expected Problems: Critical Connections in a Network, Strongly Connected Components

# Top Results:
# - Number of Connected Components in an Undirected Graph (Score: 0.923)
# - Critical Connections in a Network (Score: 0.976)
# - Minimize Malware Spread II (Score: 1.007)
# - Linked List Components (Score: 1.044)
# - Graph Valid Tree (Score: 1.058)


# Query: optimize database index structure using B+ tree with minimum memory
# Challenge Type: system_specific
# Expected Problems: Design In-Memory File System, Design Search Autocomplete System

# Top Results:
# - Closest Binary Search Tree Value II (Score: 1.043)
# - Minimum Cost Tree From Leaf Values (Score: 1.096)
# - Validate Binary Search Tree (Score: 1.113)
# - Construct Binary Tree from Inorder and Postorder Traversal (Score: 1.128)
# - Largest BST Subtree (Score: 1.145)


# Testing Edge Cases Queries:
# --------------------------------------------------

# Query: handle concurrent modifications in binary search tree with duplicates
# Challenge Type: concurrency_edge
# Expected Problems: Binary Search Tree Iterator, Serialize and Deserialize BST

# Top Results:
# - Closest Binary Search Tree Value II (Score: 0.952)
# - Recover Binary Search Tree (Score: 1.009)
# - Unique Binary Search Trees (Score: 1.011)
# - Flip Equivalent Binary Trees (Score: 1.030)
# - Unique Binary Search Trees II (Score: 1.034)


# Query: optimize space complexity for computing large fibonacci numbers with modulo
# Challenge Type: optimization_edge
# Expected Problems: Fibonacci Number, Matrix Exponentiation

# Top Results:
# - Fibonacci Number (Score: 0.865)
# - Length of Longest Fibonacci Subsequence (Score: 1.007)
# - Increasing Subsequences (Score: 1.050)
# - Count of Smaller Numbers After Self (Score: 1.063)
# - Super Pow (Score: 1.074)


# Query: handle overflow in binary tree with maximum path sum allowing negative values
# Challenge Type: numeric_edge
# Expected Problems: Binary Tree Maximum Path Sum, Path Sum II

# Top Results:
# - Binary Tree Maximum Path Sum (Score: 0.602)
# - Maximum Level Sum of a Binary Tree (Score: 0.728)
# - Insufficient Nodes in Root to Leaf Paths (Score: 0.776)
# - Path Sum II (Score: 0.784)
# - Maximum Width of Binary Tree (Score: 0.808)


# Query: find minimum cuts in flow network with negative capacity edges
# Challenge Type: constraint_edge
# Expected Problems: Network Flow, Minimum Cut, Maximum Flow

# Top Results:
# - Minimize Malware Spread II (Score: 0.973)
# - Optimize Water Distribution in a Village (Score: 1.027)
# - Critical Connections in a Network (Score: 1.063)
# - Bus Routes (Score: 1.067)
# - Connecting Cities With Minimum Cost (Score: 1.069)


# Query: optimize palindrome checking for very large strings with unicode characters
# Challenge Type: input_edge
# Expected Problems: Valid Palindrome, Longest Palindromic Substring

# Top Results:
# - Shortest Palindrome (Score: 0.790)
# - Longest Palindromic Substring (Score: 0.800)
# - Longest Chunked Palindrome Decomposition (Score: 0.805)
# - Longest Palindrome (Score: 0.834)
# - Palindrome Partitioning II (Score: 0.835)
# Testing Conceptually Ambiguous Queries (HNSW Tuned):
# --------------------------------------------------

# Query: find minimum operations to make array beautiful
# Challenge Type: conceptual_ambiguity
# Expected Problems: Minimum Operations to Make Array Beautiful, Make Array Zero by Subtracting Equal Amounts

# Top Results:
# - Shortest Subarray with Sum at Least K (Score: 0.855)
# - Sum of Subarray Minimums (Score: 0.861)
# - Minimum Size Subarray Sum (Score: 0.870)
# - Find Minimum in Rotated Sorted Array (Score: 0.888)
# - Decrease Elements To Make Array Zigzag (Score: 0.923)


# Query: optimize path finding in grid with obstacles using minimum jumps
# Challenge Type: algorithm_overlap
# Expected Problems: Jump Game, Jump Game II, Shortest Path in Binary Matrix

# Top Results:
# - Unique Paths II (Score: 0.709)
# - Unique Paths III (Score: 0.729)
# - Minimum Path Sum (Score: 0.857)
# - Minimum Moves to Reach Target with Rotations (Score: 0.875)
# - Shortest Path in Binary Matrix (Score: 0.879)


# Query: find maximum sum subarray with alternating positive negative elements
# Challenge Type: requirement_ambiguity
# Expected Problems: Maximum Subarray, Maximum Alternating Subsequence Sum

# Top Results:
# - Maximum Subarray (Score: 0.581)
# - Maximum Sum Circular Subarray (Score: 0.712)
# - Partition Array for Maximum Sum (Score: 0.726)
# - Maximum Size Subarray Sum Equals k (Score: 0.742)
# - Maximum Sum of 3 Non-Overlapping Subarrays (Score: 0.754)


# Query: optimize tree traversal with minimum memory usage without recursion
# Challenge Type: implementation_ambiguity
# Expected Problems: Binary Tree Inorder Traversal, Morris Traversal, Iterative Tree Traversal

# Top Results:
# - Binary Tree Postorder Traversal (Score: 0.732)
# - Maximum Depth of Binary Tree (Score: 0.771)
# - Binary Tree Level Order Traversal II (Score: 0.802)
# - Tree Node (Score: 0.807)
# - Increasing Order Search Tree (Score: 0.818)


# Query: find shortest cycle in undirected graph with weighted edges
# Challenge Type: algorithm_complexity
# Expected Problems: Shortest Cycle in Graph, Minimum Spanning Tree, Network Delay Time

# Top Results:
# - Shortest Path Visiting All Nodes (Score: 0.988)
# - Network Delay Time (Score: 1.040)
# - Cheapest Flights Within K Stops (Score: 1.058)
# - All Paths from Source Lead to Destination (Score: 1.068)
# - Shortest Path with Alternating Colors (Score: 1.070)


# Testing Semantically Challenging Queries:
# --------------------------------------------------

# Query: implement memcache with LRU and time-based eviction
# Challenge Type: domain_specific
# Expected Problems: LRU Cache, Design In-Memory File System, Time Based Key-Value Store

# Top Results:
# - LFU Cache (Score: 1.070)
# - LRU Cache (Score: 1.151)
# - Design Bounded Blocking Queue (Score: 1.252)
# - Building H2O (Score: 1.361)
# - Design Circular Queue (Score: 1.404)


# Query: optimize matrix chain multiplication using bottom-up approach
# Challenge Type: technical_terminology
# Expected Problems: Matrix Chain Multiplication, Burst Balloons

# Top Results:
# - Sparse Matrix Multiplication (Score: 1.064)
# - Range Addition II (Score: 1.133)
# - Longest Increasing Path in a Matrix (Score: 1.135)
# - Set Matrix Zeroes (Score: 1.154)
# - Search a 2D Matrix II (Score: 1.158)


# Query: implement concurrent rate limiter with sliding window
# Challenge Type: cross_domain
# Expected Problems: Rate Limiter, Sliding Window Maximum, Design Hit Counter

# Top Results:
# - Sliding Window Maximum (Score: 1.122)
# - Design Bounded Blocking Queue (Score: 1.130)
# - Open the Lock (Score: 1.216)
# - Moving Average from Data Stream (Score: 1.257)
# - Max Consecutive Ones III (Score: 1.271)


# Query: find strongly connected components in directed graph without Tarjan
# Challenge Type: algorithm_constraint
# Expected Problems: Critical Connections in a Network, Strongly Connected Components

# Top Results:
# - Number of Connected Components in an Undirected Graph (Score: 0.923)
# - Critical Connections in a Network (Score: 0.976)
# - Minimize Malware Spread II (Score: 1.007)
# - Linked List Components (Score: 1.044)
# - Graph Valid Tree (Score: 1.058)


# Query: optimize database index structure using B+ tree with minimum memory
# Challenge Type: system_specific
# Expected Problems: Design In-Memory File System, Design Search Autocomplete System

# Top Results:
# - Closest Binary Search Tree Value II (Score: 1.043)
# - Minimum Cost Tree From Leaf Values (Score: 1.096)
# - Validate Binary Search Tree (Score: 1.113)
# - Construct Binary Tree from Inorder and Postorder Traversal (Score: 1.128)
# - Largest BST Subtree (Score: 1.145)


# Testing Edge Cases Queries:
# --------------------------------------------------

# Query: handle concurrent modifications in binary search tree with duplicates
# Challenge Type: concurrency_edge
# Expected Problems: Binary Search Tree Iterator, Serialize and Deserialize BST

# Top Results:
# - Closest Binary Search Tree Value II (Score: 0.952)
# - Recover Binary Search Tree (Score: 1.009)
# - Unique Binary Search Trees (Score: 1.011)
# - Flip Equivalent Binary Trees (Score: 1.030)
# - Unique Binary Search Trees II (Score: 1.034)


# Query: optimize space complexity for computing large fibonacci numbers with modulo
# Challenge Type: optimization_edge
# Expected Problems: Fibonacci Number, Matrix Exponentiation

# Top Results:
# - Fibonacci Number (Score: 0.865)
# - Length of Longest Fibonacci Subsequence (Score: 1.007)
# - Increasing Subsequences (Score: 1.050)
# - Count of Smaller Numbers After Self (Score: 1.063)
# - Super Pow (Score: 1.074)


# Query: handle overflow in binary tree with maximum path sum allowing negative values
# Challenge Type: numeric_edge
# Expected Problems: Binary Tree Maximum Path Sum, Path Sum II

# Top Results:
# - Binary Tree Maximum Path Sum (Score: 0.602)
# - Maximum Level Sum of a Binary Tree (Score: 0.728)
# - Insufficient Nodes in Root to Leaf Paths (Score: 0.776)
# - Path Sum II (Score: 0.784)
# - Maximum Width of Binary Tree (Score: 0.808)


# Query: find minimum cuts in flow network with negative capacity edges
# Challenge Type: constraint_edge
# Expected Problems: Network Flow, Minimum Cut, Maximum Flow

# Top Results:
# - Minimize Malware Spread II (Score: 0.973)
# - Optimize Water Distribution in a Village (Score: 1.027)
# - Critical Connections in a Network (Score: 1.063)
# - Bus Routes (Score: 1.067)
# - Connecting Cities With Minimum Cost (Score: 1.069)


# Query: optimize palindrome checking for very large strings with unicode characters
# Challenge Type: input_edge
# Expected Problems: Valid Palindrome, Longest Palindromic Substring

# Top Results:
# - Shortest Palindrome (Score: 0.790)
# - Longest Palindromic Substring (Score: 0.800)
# - Longest Chunked Palindrome Decomposition (Score: 0.805)
# - Longest Palindrome (Score: 0.834)
# - Palindrome Partitioning II (Score: 0.835)

NameError: name '__file__' is not defined